## RAISS python version verification

## Overview:
    
Since the RAISS R code is derived from original python code at https://gitlab.pasteur.fr/statistical-genetics/raiss/-/blob/master/raiss/stat_models.py,
To verify that our R code is correct, I use python version to reimplement the result. 

## Data: 

The data is from real data of a region with 5565 SNPs, 13 SNPs don't have z scores but LD matrix is available, we try to use RAISS to impute the 13 missing SNPs z score.

zscores: `~/RSS_QC/data/python_compare/zt.csv` dimension 5565 * 1
LD of known SNPs : `~/RSS_QC/data/python_compare/sig_t.csv`, dimension 5565 * 5565
LD of known SNPs and unknown SNPs : `~/RSS_QC/data/python_compare/sig_t.csv`, dimension 13 * 5565


## Conclusion:

The result using raiss_model function is totally the same. Our R code is correct.

In [43]:
"""
This module contain the statistical library for imputation.

Notation style of matrices subset and vectors are based on the publication:

Bogdan Pasaniuc, Noah Zaitlen, Huwenbo Shi, Gaurav Bhatia, Alexander Gusev,
Joseph Pickrell, Joel Hirschhorn, David P. Strachan, Nick Patterson,
Alkes L. Price;
Fast and accurate imputation of summary statistics enhances evidence
of functional enrichment, Bioinformatics, Volume 30, Issue 20, 15 October 2014,
Pages 2906–2914

"""

import numpy as np
import scipy as sc
import scipy.linalg

def compute_mu(sig_i_t, sig_t_inv, zt):
    """
    Compute the estimation of z-score from neighborring snp

    Args:
        sig_i_t (matrix?) : correlation matrix with line corresponding to
        unknown Snp (snp to impute) and column to known SNPs
        sig_t_inv (np.ndarray): inverse of the correlation matrix of known
        matrix
        zt (np.array?): Zscores of known snp
    Returns:
        mu_i (np.array): a vector of length i containing the estimate of zscore

    """
    return np.dot(sig_i_t, np.dot(sig_t_inv, zt))

def compute_var(sig_i_t, sig_t_inv, lamb, batch=True):
    """
    Compute the expected variance of the imputed SNPs
    Args:
        sig_i_t (matrix?) : correlation matrix with line corresponding to
        unknown Snp (snp to impute) and column to known SNPs
        sig_t_inv (np.ndarray): inverse of the correlation matrix of known
        matrix
        lamb (float): regularization term added to matrix

    """

    if batch:
        var = (1 + lamb) - np.einsum('ij,jk,ki->i', sig_i_t, sig_t_inv ,sig_i_t.transpose())
        ld_score = (sig_i_t**2).sum(1)
    else:
        var = (1 + lamb) - np.dot(sig_i_t, np.dot(sig_t_inv, sig_i_t.transpose()))
        ld_score = (sig_i_t**2).sum()
    return var, ld_score

def check_inversion(sig_t, sig_t_inv):
    return np.allclose(sig_t, np.dot(sig_t, np.dot(sig_t_inv, sig_t)))

def var_in_boundaries(var,lamb):
    """
    Forces the variance to be in the 0 to 1+lambda boundary
    theoritically we shouldn't have to do that
    """
    id_neg = np.where(var < 0)
    var_norm = var
    var[id_neg] = 0
    id_inf = np.where(var > (0.99999+lamb))
    var[id_inf] = 1

    return var

def invert_sig_t(sig_t, lamb, rtol):
    try:
        np.fill_diagonal(sig_t, (1+lamb))
        sig_t_inv = scipy.linalg.pinv(sig_t, rtol=rtol,atol=0)
        return(sig_t_inv)
    except np.linalg.LinAlgError:
        invert_sig_t(sig_t, lamb*1.1, rtol*1.1)

def raiss_model(zt, sig_t, sig_i_t, lamb=0.01, rtol=0.01,  batch=True):
    """
    Compute the variance
    Args:
        zt (np.array): the vector of known Z scores
        sig_t (np.ndarray) : the matrix of known Linkage desiquilibrium
         correlation
        sig_i_t (np.ndarray): correlation matrix of known matrix
        lamb (float): regularization term added to the diagonal of the sig_t matrix
        rtol (float): threshold to filter eigenvector with a eigenvalue under rtol
        make inversion biased but much more numerically robust
    """
    sig_t_inv = invert_sig_t(sig_t, lamb, rtol)
    if sig_t_inv is None:
        return None
    else:
        if batch:
            condition_number = np.array([np.linalg.cond(sig_t)]*sig_i_t.shape[0])
            correct_inversion = np.array([check_inversion(sig_t, sig_t_inv)]*sig_i_t.shape[0])
        else:
            condition_number = np.linalg.cond(sig_t)
            correct_inversion = check_inversion(sig_t, sig_t_inv)
        var, ld_score = compute_var(sig_i_t, sig_t_inv, lamb, batch)

        mu = compute_mu(sig_i_t, sig_t_inv, zt)
        var_norm = var_in_boundaries(var, lamb)

        R2 = ((1+lamb)-var_norm)

        mu = mu / np.sqrt(R2)
        return({"var" : var, "mu" : mu, "ld_score" : ld_score, "condition_number" : condition_number, "correct_inversion":correct_inversion })


In [49]:
import pandas as pd
import numpy as np

# Read CSV file into a pandas DataFrame
zt = pd.read_csv("/home/hs3393/RSS_QC/data/python_compare/zt.csv", header=None)
sig_t = pd.read_csv("/home/hs3393/RSS_QC/data/python_compare/sig_t.csv", header=None)
sig_i_t = pd.read_csv("/home/hs3393/RSS_QC/data/python_compare/sig_i_t.csv", header=None)



In [77]:
# Convert DataFrame to NumPy ndarray
zt = zt.to_numpy()
sig_t = sig_t.to_numpy()
sig_i_t = sig_i_t.to_numpy()

zt = zt.T

res = raiss_model(zt, sig_t, sig_i_t, lamb=0.01, rtol=0.01,  batch=True)

In [78]:
res

{'var': array([0.01560217, 0.02432745, 0.21867454, 0.03558347, 0.19160761,
        0.06203065, 0.09720293, 0.03220463, 0.248352  , 0.64194037,
        0.02318182, 0.04403999, 0.05428655]),
 'mu': array([-3.80199571, -2.69057991,  3.93713591,  6.12016482,  4.47129728,
         5.73833363,  5.29888995,  2.29405333, -3.60477839,  0.86846564,
        -3.99102852,  2.18083374,  2.5667356 ]),
 'ld_score': array([66.06321894, 41.80978648, 16.75226928, 47.68584184, 16.96760059,
        39.55272632, 35.21468757, 52.61507398, 20.40925093,  3.80182507,
        55.41959595, 49.06049168, 42.36681352]),
 'condition_number': array([192331.259663, 192331.259663, 192331.259663, 192331.259663,
        192331.259663, 192331.259663, 192331.259663, 192331.259663,
        192331.259663, 192331.259663, 192331.259663, 192331.259663,
        192331.259663]),
 'correct_inversion': array([False, False, False, False, False, False, False, False, False,
        False, False, False, False])}

# Compare with R result

In [4]:

library(tidyverse)
source("/home/hs3393/RSS_QC/pecotmr/R/raiss.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [11]:
zt = read.csv("/home/hs3393/RSS_QC/data/python_compare/zt.csv", header = F) %>% as.matrix()
sig_t = read.csv("/home/hs3393/RSS_QC/data/python_compare/sig_t.csv", header = F) %>% as.matrix()
sig_i_t = read.csv("/home/hs3393/RSS_QC/data/python_compare/sig_i_t.csv", header = F) %>% as.matrix()

In [16]:
dim(zt)
dim(sig_t)
dim(sig_i_t)

[1] 5565    1

[1] 5565 5565

[1]   13 5565

In [14]:
raiss_model(zt, sig_t, sig_i_t)

V1
-3.8019957
-2.6905799
3.9371359
6.1201648
4.4712973
5.7383336
5.2988900
2.2940533
-3.6047784
0.8684656


By simply checking, all outputs are totally the same except for the conditionalnumber. But it seems like it'v not involved in the output. The rest part of RAISS function is to make it a dataframe. So our transfering from python to R is valid.